In [371]:
import keras
import numpy as np
import lstm

In [ ]:
import numpy as np
from numpy.typing import NDArray
from typing import Callable, Tuple
from keras.src.backend import Variable
import keras

class LSTMScratch():
    def __init__(
            self,
            units: int, # neuron
            keras_weight: list[Variable],
            activation: Callable[[NDArray[np.float64]], NDArray[np.float64]] = np.tanh, # activation function
    ):
        # self.input_feature = input_feature
        self.kernel, self.recurrent_kernel, self.bias = keras_weight
        self.h_t = [np.zeros(units)]
        self.c_t = [np.zeros(units)]
        self.activation = activation

    def predict(self, input_feature):
        timestep = input_feature.shape[1]
        sample = input_feature.shape[0]
        for i in range(sample):
            current_sample = input_feature[i]
            temp_h = []
            temp_c = []
            for j in range(timestep):
                h_t, c_t = self.calc(current_sample[j]) 
                temp_h.append(h_t)
                temp_c.append(c_t)
            self.h_t.append(temp_h)
            self.h_t.append(temp_c)
            
        return self.h_t[-1]

    def calc(self, x_t: NDArray[np.float64]):
        w_dot_x = x_t @ self.kernel
        u_dot_h = self.h_t[-1] @ self.recurrent_kernel
        result = w_dot_x + u_dot_h + self.bias 
        k_i, k_f, k_c, k_o = np.split(result, 4, axis=0)

        i_t, f_t, o_t = self.sigmoid(k_i), self.sigmoid(k_f), self.sigmoid(k_o)

        c_tilde = self.activation(k_c)
        c_t = f_t * self.c_t[-1] + i_t * c_tilde
        
        h_t = o_t * self.activation(c_t)

        return h_t, c_t     

    def sigmoid(self, x):
        return 1/(1+np.exp(-x))


In [373]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(5))

In [374]:
X = np.random.rand(100, 10, 2)
y = np.random.rand(100, 1)

In [375]:
model.compile(optimizer="adam", loss="mse")
model.fit(X, y, epochs=5, batch_size=16)
pred = model.predict(X[:5])
print(pred)

Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.4525
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.4393
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.4148
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.3918
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.3866
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step
[[-0.17868511 -0.0073087   0.04492623 -0.2070542   0.33834067]
 [-0.1722766   0.08218046 -0.07089911 -0.27539545  0.35626215]
 [-0.11461253  0.03432412 -0.02614582 -0.19594355  0.33535776]
 [-0.15979825  0.03069684 -0.00911366 -0.2144333   0.32882914]
 [-0.18552595 -0.06563866  0.10684092 -0.26994315  0.4476928 ]]


In [376]:
# model.summary()
# model.layers
# for i in model.layers:
#     for j in i.weights:
#         print(type(j))

In [377]:
weight = model.get_weights()
print(model.weights)

[<Variable path=sequential_37/lstm_37/lstm_cell/kernel, shape=(2, 20), dtype=float32, value=[[ 0.08214381 -0.00814943  0.02216865 -0.18522549  0.3427284  -0.17373887
  -0.23386706  0.48221728  0.00997036 -0.39361897 -0.163952    0.19975011
  -0.3456993  -0.4275032   0.37185118 -0.0607361   0.3855842   0.04948305
   0.43183735  0.3125835 ]
 [-0.07338985 -0.04773314  0.3708444  -0.4353899   0.23260866  0.29511324
  -0.44118047  0.2067154  -0.02895414  0.14113814 -0.23774779 -0.40696552
   0.3884269  -0.4737121   0.5098203  -0.5450622   0.03170464  0.18599223
  -0.23330691  0.2636408 ]]>, <Variable path=sequential_37/lstm_37/lstm_cell/recurrent_kernel, shape=(5, 20), dtype=float32, value=[[-0.13173716  0.12096208  0.30459633 -0.13839608 -0.51484233 -0.13614663
  -0.0721954  -0.24811144  0.08031714 -0.0181589  -0.03040656 -0.08328062
  -0.28515884  0.14050317  0.11976752  0.06207029  0.16015942  0.01128154
  -0.12045991  0.56238437]
 [-0.00713899  0.01314056 -0.18692902  0.47736585 -0.1869

In [378]:
arr = np.array([[1,2], [3,4], [9, 9]])
x = LSTMScratch(5, weight)

In [379]:
x.predict(X[:5])
# model.predict(arr)

array([-0.19015739, -0.0630859 ,  0.10618754, -0.27462613,  0.45055029])

In [ ]:
array([-0.19015739, -0.0630859 ,  0.10618754, -0.27462613,  0.45055029])

[[-0.17868511 -0.0073087   0.04492623 -0.2070542   0.33834067]
 [-0.1722766   0.08218046 -0.07089911 -0.27539545  0.35626215]
 [-0.11461253  0.03432412 -0.02614582 -0.19594355  0.33535776]
 [-0.15979825  0.03069684 -0.00911366 -0.2144333   0.32882914]
 [-0.18552595 -0.06563866  0.10684092 -0.26994315  0.4476928 ]]